In [1]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# -------------------------------------------------------------------
# 1. GP + Sigmoid 混合模型
# -------------------------------------------------------------------

# Sigmoid 部分 (你原始的理论模型形式)
def sigmoid_v(P, T, W):
    return 1 / (1 + np.exp(-0.01 * (T - 100))) * \
           (1 / (1 + np.exp(-0.1 * (P - 32)))) * 3

def sigmoid_a(W):
    return 1 + 2 / (1 + np.exp(-0.01 * (W - 600)))


# -------------------------------------------------------------------
# 2. 训练 GP 模型（用于生成灵活非线性部分）
# -------------------------------------------------------------------

# 构建训练点，用于定义 GP 的趋势（可简单随机）
def generate_gp_training_data(n_points=40):
    P = np.random.uniform(1, 60, n_points)
    T = np.random.uniform(50, 300, n_points)
    W = np.random.uniform(200, 1000, n_points)

    X = np.column_stack([P, T, W])

    # 目标值 = sigmoid 输出 + 噪声
    y = sigmoid_v(P, T, W) + np.random.normal(0, 0.1, n_points)

    return X, y

# 训练 GP
X_train, y_train = generate_gp_training_data()
kernel = 1.0 * RBF(length_scale=50.0) + WhiteKernel(noise_level=0.01)
gp_v = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(X_train, y_train)


# -------------------------------------------------------------------
# 3. 混合模型（Sigmoid + GP）
# -------------------------------------------------------------------

# 混合权重
w = 0.5   # ← 可调整：越大越偏理论 Sigmoid；越小越偏 GP

def compute_v(P, T, W):
    X = np.array([[P, T, W]])
    v_gp = gp_v.predict(X)[0]
    v_sig = sigmoid_v(P, T, W)
    return w * v_sig + (1 - w) * v_gp

def compute_a(P, T, W):
    a_sig = sigmoid_a(W)
    a_gp = 1.5 + 0.1 * np.sin(P / 10) + 0.1 * np.cos(W / 200)
    return w * a_sig + (1 - w) * a_gp


# -------------------------------------------------------------------
# 4. DDM Euler 轨迹模拟
# -------------------------------------------------------------------

def simulate_ddm(v, a, t0=0.2, dt=0.001):
    x = 0
    upper = a
    lower = -a
    t = 0
    noise = np.sqrt(dt) * 1.0

    while True:
        x += v * dt + noise * np.random.randn()
        t += dt

        if x >= upper:
            return t + t0, 1
        if x <= lower:
            return t + t0, 0


# -------------------------------------------------------------------
# 5. 数据生成
# -------------------------------------------------------------------

def generate_subject_data(subject_id, n_trials=200):
    results = []

    # Self=1 与 Other=0 必须数量相同
    labels = np.array([1] * (n_trials // 2) + [0] * (n_trials // 2))
    np.random.shuffle(labels)

    for trial in range(n_trials):
        label = labels[trial]

        # 随机生成实验变量
        P = np.random.randint(1, 60)
        T = np.random.uniform(50, 300)
        W = np.random.uniform(200, 1000)

        # Matching 随机
        M = np.random.randint(0, 2)

        # v & a
        v = compute_v(P, T, W)
        if label == 1:
            v *= 1.3
        else:
            v *= 0.8

        a = compute_a(P, T, W)

        # 固定参数
        t0 = 0.2
        z = 2 / a

        # 运行 DDM
        RT, response = simulate_ddm(v, a, t0=t0)

        # 保存
        results.append([
            subject_id, trial + 1, P, T, W, M, label,
            v, a, t0, z, RT, response
        ])

    return results


# -------------------------------------------------------------------
# 6. 多被试数据生成 + 保存为 CSV
# -------------------------------------------------------------------

all_data = []

n_subjects = 5
trials_per_subject = 260

for s in range(1, n_subjects + 1):
    data = generate_subject_data(s, n_trials=trials_per_subject)
    all_data.extend(data)

columns = [
    "subject", "trial", "P", "T", "W", "M", "label",
    "v", "a", "t0", "z", "RT", "response"
]

df = pd.DataFrame(all_data, columns=columns)

df.to_csv("ddm_sigmoid_gp_hybrid.csv", index=False)

print("CSV 文件已保存为：ddm_sigmoid_gp_hybrid.csv")
print(df.head())


CSV 文件已保存为：ddm_sigmoid_gp_hybrid.csv
   subject  trial   P           T           W  M  label         v         a  \
0        1      1  18  253.824952  345.579580  0      0  0.455621  1.363686   
1        1      2  35   89.925362  999.658231  1      0  0.640861  2.228515   
2        1      3  11  299.322722  214.697421  0      1  0.519269  1.339188   
3        1      4  11  169.912333  726.668528  1      0  0.193382  2.030660   
4        1      5  51  160.097978  466.426744  0      0  1.319093  1.377427   

    t0         z     RT  response  
0  0.2  1.466613  2.297         1  
1  0.2  0.897458  7.467         1  
2  0.2  1.493442  0.801         1  
3  0.2  0.984902  3.590         0  
4  0.2  1.451982  1.566         1  
